In [1]:
import requests
import json
import pandas as pd
from tmdbv3api import Movie, TMDb

## Coleta de dados

1ª base de dados:
Esta base será coletada de um site que possui um API que cadastrado uma chave consegue-se extrair dados de um filme através do id do filme.
O site The Movie Database, mais conhecido pela sigla TMDb, é uma base de dados grátis e de código aberto, sobre filmes e seriados, criado por Travis Bell em 2008. 

API de acesso a dados de filmes: https://developers.themoviedb.org/4/getting-started/authorization
Site para cadastro de chave api key: https://www.themoviedb.org/

In [77]:
# teste de acesso
MOVIE_ID = 561362
API_KEY = 'd2dee50252ec42a906562d4f3c43dfb0'

In [102]:
url = f'https://api.themoviedb.org/3/movie/{MOVIE_ID}?api_key={API_KEY}'

In [103]:
# verificando o acesso a url
response = requests.get(url)
print(response)

<Response [200]>


In [105]:
# retornando os dados da API e transformando no formato json
dict_text = json.loads(response.text)

# extraindo as chaves dos filmes para posteriormente se tornarem as variaveis do dataframe
dict_text.keys()

dict_keys(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count'])

Para extrair as informações precisamos do id dos filmes que queremos, que serão filmes lançados em 2019 a 2021 nesta base de dados, assim vamos utilizar o módulo tmdbv3api (link de informações abaixo) para acessar a api e trazer os ids dos filmes.
https://github.com/AnthonyBloomer/tmdbv3api

In [39]:
pip install tmdbv3api


Note: you may need to restart the kernel to use updated packages.


In [115]:
# instanciando a classe para dar acesso a API com a chave
tmdb = TMDb()
tmdb.api_key = 'd2dee50252ec42a906562d4f3c43dfb0'

In [44]:
# filmes de 2019
url_2019 = 'https://en.wikipedia.org/wiki/List_of_American_films_of_2019'
response = requests.get(url_2019)
print(response)

<Response [200]>


Vamos formar um dataframe com os filmes dos anos 2019,2020,2021 coletando a partir de uma lista no wikipedia.

In [223]:
anos = [2019,2020,2021]

df_geral = pd.DataFrame()


for i in range(len(anos)):
    
    try:
        
        # url do wikipedia para onde contem a lista de filmes
        url = f'https://en.wikipedia.org/wiki/List_of_American_films_of_{anos[i]}'
        response = requests.get(url)
        
        # os indices trazem os dados dos filmes a cada 3 meses, por isso são 4 dataframes para o ano
        df1 = pd.read_html(url, header=0)[2]
        df2 = pd.read_html(url, header=0)[3]
        df3 = pd.read_html(url, header=0)[4]
        df4 = pd.read_html(url, header=0)[5]
        
        # juntando os dataframes
        df = df1.append(df2.append(df3.append(df4, ignore_index = False)))
        
        df = df['Title']
        
        colunas = [df_geral, df]
        df_geral = pd.concat(colunas)
        
    except:
        continue

C:\Users\prpau\AppData\Local\Temp\ipykernel_12032\3210825781.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4, ignore_index = False)))
C:\Users\prpau\AppData\Local\Temp\ipykernel_12032\3210825781.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4, ignore_index = False)))
C:\Users\prpau\AppData\Local\Temp\ipykernel_12032\3210825781.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4, ignore_index = False)))
C:\Users\prpau\AppData\Local\Temp\ipykernel_12032\3210825781.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

In [224]:
df_geral.head()

,0
0,Escape Room
1,Rust Creek
2,American Hangman
3,A Dog's Way Home
4,The Upside


In [225]:
df_geral.rename(columns = {0: 'Title'}, inplace = True)
df_geral.head()

,Title
0,Escape Room
1,Rust Creek
2,American Hangman
3,A Dog's Way Home
4,The Upside


Vamos incluir o id no dataset através de uma função que instancia uma classe do módulo tmdbv3api

In [227]:
# função que retorna o id atraves do nome do filme
def get_id_movie (title):
    try:
        
        tmdb_movie = Movie()
        movie = tmdb_movie.search(title)
        return movie[0].id
    except:
        return None
    

In [228]:
# incluindo a com id dos filmes coletados na API pela função get_id_movie
df_geral['id'] = df_geral['Title'].map(lambda x: get_id_movie(str(x)))

In [229]:
len(df_geral)

881

In [234]:
df_geral['id']

0     522681.0
1     561362.0
2     567738.0
3     508763.0
4     440472.0
        ...   
93    591538.0
94    501841.0
95    673309.0
96    511819.0
97    285305.0
Name: id, Length: 881, dtype: float64

In [235]:
# separando as colunas para coletar os dados
lista_colunas = list(dict_text.keys())
lista_colunas.remove('id')
lista_colunas.remove('imdb_id')
lista_colunas.remove('title')

lista_colunas


['adult',
 'backdrop_path',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'video',
 'vote_average',
 'vote_count']

In [239]:
# função que retorna os dados da coluna solicitada na API atraves do filme buscado

def get_columns(column, movie):
    try:
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie, tmdb.api_key))
        data_j = response.json()
        return data_j[column]
    except:
        return None



In [240]:
# aplicação da função get_columns e adicionando ao dataframe

for i in lista_colunas:
    
    df_geral[i] = df_geral['id'].map(lambda x: get_columns(i, x))

In [241]:
df_geral.head()

,Title,id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,video,vote_average,vote_count
0,Escape Room,522681.0,False,/mWLljCmpqlcYQh7uh51BBMwCZwN.jpg,"{'id': 769423, 'name': 'Escape Room Collection...",9000000.0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",https://www.escaperoom.movie/,en,Escape Room,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2019-01-03,155712077.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Find The Clues Or Die,False,6.544,3961.0
1,Rust Creek,561362.0,False,/xmLLkEhqIw0Me0wgO9ZhtP0MW7U.jpg,None,750000.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://www.rustcreek.com,en,Rust Creek,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2019-01-04,0.0,108.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Choose your prey carefully,False,6.409,248.0
2,American Hangman,567738.0,False,/1iUuWGnywpwgcoHQ1LS6TzXtqLt.jpg,None,0.0,"[{'id': 53, 'name': 'Thriller'}]",,en,American Hangman,...,"[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2019-01-04,0.0,99.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,A new kind of trial,False,6.100,120.0
3,A Dog's Way Home,508763.0,False,/w7MsyOpP003FYhsiTgVHa3rNsrz.jpg,None,18000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 12, 'name...",http://www.adogswayhome.movie/,en,A Dog's Way Home,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2019-01-10,17643857.0,96.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,A lot can happen between lost and found.,False,7.309,941.0
4,The Upside,440472.0,False,/4lQQVRLc7OCEFH6OUaMztwiFPbQ.jpg,None,37500000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",http://www.theupside.movie/,en,The Upside,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2019-01-10,111353135.0,126.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Based on a true story.,False,7.117,1026.0


In [242]:
df_geral.shape

(881, 24)

In [243]:
df_geral.to_csv('filmes.csv', sep = ';', index = False)

### 2ª base de dados

A segunda base de dados esta no kaggle: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata. 

Dataset do kaggle para extrair
https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?select=movies_metadata.csv

In [265]:
df_movies = pd.read_csv('tmdb_5000_movies.csv/tmdb_5000_movies.csv')
df_movies.shape, df_geral.shape

((4803, 20), (881, 24))

Há uma diferença nas colunas, vamos verificar quais.

In [267]:

for column in df_geral:
    
    if column in df_movies.columns:
        pass
    else:
        print(column)

Title
adult
backdrop_path
belongs_to_collection
poster_path
video


Das colunas diferentes, vamos permanecer apenas com Title (titulo do filme, o df_movie também possui esta variavel, mas com letras todas em minúsculas), e adult( se o filme é para adultos), as outras variaveis são endereços de imagens ou possuem muitos valores faltantes, então serão descartadas.

In [280]:
for column in df_movies:
    
    if column in df_geral.columns:
        pass
    else:
        print(column)

keywords
title


Para que os datasets fiquem iguais nas variáveus vamos coletar keywords para inserir no df_geral e adult para inserir no df_movies antes de realizar o merge.

In [278]:
df_geral.drop(['backdrop_path', 'belongs_to_collection', 'poster_path', 'video'], axis = 1, inplace = True)
df_geral.columns

Index(['Title', 'id', 'adult', 'budget', 'genres', 'homepage',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline',
       'vote_average', 'vote_count'],
      dtype='object')

In [284]:
df_geral.rename(columns = {'Title': 'title'}, inplace = True)
df_geral.columns

Index(['title', 'id', 'adult', 'budget', 'genres', 'homepage',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline',
       'vote_average', 'vote_count'],
      dtype='object')

In [286]:
def get_adult(movie):
    
    try:
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie, tmdb.api_key))
        data_j = response.json()
        return data_j['adult']
    
    except:
        return None
    

def get_keywords(movie):
    
    try:
        response = requests.get('https://api.themoviedb.org/3/movie/{}/keywords?api_key={}'.format(movie, tmdb.api_key))
        data_j = response.json()
        return data_j['keywords']
    
    except:
        return None

In [ ]:
df_geral['keywords'] = df_geral['id'].map(lambda x: get_keywords(str(x)))
df_movies['adult'] = df_movies['id'].map(lambda x: get_adult(str(x)))

In [291]:
dataframes = [df_geral, df_movies]
df_final = pd.concat(dataframes)
df_final.shape

(5684, 21)

In [295]:
df_final.to_csv('filmes_completo.csv', sep = ';', index = False)

In [ ]:
######################################################

In [296]:
'''
def get_credits_crew(movie):
    
    try:
        response = requests.get('https://api.themoviedb.org/3/movie/{}/credits?api_key={}'.format(movie, tmdb.api_key))
        data_j = response.json()
        return data_j['crew']
    
    except:
        return None

def get_credits_cast(movie):
    
    try:
        response = requests.get('https://api.themoviedb.org/3/movie/{}/credits?api_key={}'.format(movie, tmdb.api_key))
        data_j = response.json()
        return data_j['cast']
    
    except:
        return None

In [ ]:
'''
df_final['crew'] = df_final['id'].map(lambda x: get_credits_crew(str(x)))
df_final['cast'] = df_final['id'].map(lambda x: get_credits_cast(str(x)))

### 3 base de dados
Esta base possui dados de cast e crew que são elenco e todas as outras pessoas que trabalham no filme (produção, diretior, etc). Esta base também esta no kaggle: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

In [261]:
df_creditos = pd.read_csv("tmdb_5000_credits.csv/tmdb_5000_credits.csv")
df_creditos.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [306]:
# renomear a coluna de id para ficar igual ao df_final
df_creditos.rename(columns = {'movie_id':'id'}, inplace = True)

# este dataset ficará somente com as colunas de interesse e o id para o merge
df_creditos.drop(columns = ['title'], inplace = True)
df_creditos.head()

,id,cast,crew
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [300]:
df_creditos.shape

(4803, 4)

In [257]:
try:
    df_geral.id.astype('int64')
except:
    None
df_geral.id.dtype

dtype('float64')

In [313]:
df_tmdb = df_final.merge(df_creditos, on = 'id')

In [314]:
df_tmdb.shape

(4819, 23)

In [316]:
df_tmdb.to_csv('tmdb_movies_full.csv', sep = ';', index = False)

## Análise exploratória dos dados (EDA)

In [2]:
df_tmdb = pd.read_csv('tmdb_movies_full.csv', sep = ";")

In [3]:
df_tmdb.head()

,title,id,adult,budget,genres,homepage,original_language,original_title,overview,popularity,...,revenue,runtime,spoken_languages,status,tagline,vote_average,vote_count,keywords,cast,crew
0,Serenity,16320.0,False,39000000.0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",NaN,en,Serenity,When the renegade crew of Serenity agrees to h...,21.853000,...,38869464.0,119.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They aim to misbehave.,7.381,2981.0,"[{'id': 779, 'name': 'martial arts'}, {'id': 1...","[{""cast_id"": 3, ""character"": ""Mal"", ""credit_id...","[{""credit_id"": ""52fe46c59251416c75082fe3"", ""de..."
1,Serenity,16320.0,False,39000000.0,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",NaN,en,Serenity,When the renegade crew of Serenity agrees to h...,45.388937,...,38869464.0,119.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Can't stop the signal.,7.400,1264.0,"[{""id"": 779, ""name"": ""martial arts""}, {""id"": 1...","[{""cast_id"": 3, ""character"": ""Mal"", ""credit_id...","[{""credit_id"": ""52fe46c59251416c75082fe3"", ""de..."
2,The Kid,4244.0,False,65000000.0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 35, 'na...",http://movies.disney.com/the-kid,en,The Kid,Powerful businessman Russ Duritz is self-absor...,24.648000,...,110317580.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Nobody ever grows up quite like they imagined.,6.474,802.0,"[{'id': 965, 'name': 'age difference'}, {'id':...","[{""cast_id"": 1, ""character"": ""Russell Duritz"",...","[{""credit_id"": ""52fe43b5c3a36847f806964b"", ""de..."
3,The Kid,4244.0,False,65000000.0,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 35, ""na...",http://movies.disney.com/the-kid,en,The Kid,Powerful businessman Russ Duritz is self-absor...,18.281798,...,69700000.0,104.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Nobody ever grows up quite like they imagined.,6.000,238.0,"[{""id"": 965, ""name"": ""age difference""}, {""id"":...","[{""cast_id"": 1, ""character"": ""Russell Duritz"",...","[{""credit_id"": ""52fe43b5c3a36847f806964b"", ""de..."
4,Pet Sematary,8913.0,False,11500000.0,"[{'id': 27, 'name': 'Horror'}, {'id': 18, 'nam...",NaN,en,Pet Sematary,Dr. Louis Creed's family moves into the countr...,36.157000,...,57469467.0,103.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Sometimes dead is better.,6.609,1768.0,"[{'id': 818, 'name': 'based on novel or book'}...","[{""cast_id"": 10, ""character"": ""Louis Creed"", ""...","[{""credit_id"": ""52fe44c7c3a36847f80a989b"", ""de..."


In [4]:
df_tmdb.isnull().sum()

title                      0
id                         0
adult                      7
budget                     0
genres                     0
homepage                3100
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  845
vote_average               0
vote_count                 0
keywords                   0
cast                       0
crew                       0
dtype: int64

In [8]:
df_tmdb.tagline

0                                  They aim to misbehave.
1                                  Can't stop the signal.
2          Nobody ever grows up quite like they imagined.
3          Nobody ever grows up quite like they imagined.
4                               Sometimes dead is better.
                              ...                        
4814    He didn't come looking for trouble, but troubl...
4815    A newlywed couple's honeymoon is upended by th...
4816                                                  NaN
4817                             A New Yorker in Shanghai
4818                                                  NaN
Name: tagline, Length: 4819, dtype: object